In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

def combine_congress_csvs():
    """
    Combine House and Senate/JointRes CSVs
    
    Expected file names:
    - ../data/bills_118th_congress_house.csv
    - ../data/bills_118th_congress_senate_joint.csv
    """
    
    print("Combining House and Senate/Joint Resolution bills...")
    
    # Load the CSVs
    house_df = pd.read_csv('bills_118th_congress_house.csv')
    senate_joint_df = pd.read_csv('bills_118th_congress_senate_joint.csv')
    
    print(f"House bills: {len(house_df)}")
    print(f"Senate/Joint bills: {len(senate_joint_df)}")
    
    # Combine them
    combined_df = pd.concat([house_df, senate_joint_df], ignore_index=True)
    
    print(f"Total combined bills: {len(combined_df)}")
    
    # Save combined dataset
    combined_df.to_csv('bills_118th_congress_full.csv', index=False)
    
    return combined_df

def identify_dead_bills(df):
    """
    Identify bills that died in committee and mark them as failed
    
    Since the 118th Congress has ended (January 2025), ANY bill that didn't pass is now FAILED.
    """
    
    print("\n" + "="*60)
    print("IDENTIFYING FAILED BILLS FROM ENDED 118TH CONGRESS")
    print("="*60)
    
    # Convert dates
    df['introduced_date'] = pd.to_datetime(df['introduced_date'], errors='coerce')
    df['latest_action_date'] = pd.to_datetime(df['latest_action_date'], errors='coerce')
    
    # Current date
    current_date = datetime.now()
    congress_end_date = datetime(2025, 1, 3)
    
    print(f"Current date: {current_date.strftime('%Y-%m-%d')}")
    print(f"118th Congress ended: {congress_end_date.strftime('%Y-%m-%d')}")
    print(f"Days since Congress ended: {(current_date - congress_end_date).days}")
    
    # Original outcome distribution
    print(f"\nOriginal outcome distribution:")
    print(f"Passed: {(df['passed'] == 1).sum()}")
    print(f"Failed: {(df['passed'] == 0).sum()}")
    print(f"Pending: {(df['passed'] == -1).sum()}")
    
    # CRITICAL: Since Congress has ended, ALL pending bills are now FAILED
    print("\n🔴 118th Congress has ended - all pending bills are now FAILED")
    
    # Mark ALL pending bills as failed
    pending_bills = df['passed'] == -1
    df.loc[pending_bills, 'passed'] = 0
    
    # Categorize failure reasons based on how far they got
    print("\nCategorizing failure reasons...")
    
    # Bills that never left committee
    never_left_committee = pending_bills & (
        (df['action_count'] <= 2) |
        (df['latest_action'].str.contains('referred to', case=False, na=False))
    )
    df.loc[never_left_committee, 'failure_reason'] = 'died_in_committee'
    
    # Bills with some progress but didn't pass
    some_progress = pending_bills & (
        (df['action_count'] > 2) & 
        (df['action_count'] <= 10) &
        (~never_left_committee)
    )
    df.loc[some_progress, 'failure_reason'] = 'stalled_in_process'
    
    # Bills that got far but didn't make it
    significant_progress = pending_bills & (
        (df['action_count'] > 10) |
        (df['latest_action'].str.contains('passed', case=False, na=False))
    ) & (~some_progress) & (~never_left_committee)
    df.loc[significant_progress, 'failure_reason'] = 'failed_to_complete'
    
    # Any remaining pending bills
    remaining_pending = pending_bills & (df['failure_reason'].isna())
    df.loc[remaining_pending, 'failure_reason'] = 'congress_ended'
    
    # Final outcome distribution
    print(f"\nFinal outcome distribution:")
    print(f"Passed: {(df['passed'] == 1).sum()} ({(df['passed'] == 1).mean()*100:.1f}%)")
    print(f"Failed: {(df['passed'] == 0).sum()} ({(df['passed'] == 0).mean()*100:.1f}%)")
    print(f"Pending: {(df['passed'] == -1).sum()} (should be 0)")
    
    # Verify no pending bills remain
    if (df['passed'] == -1).sum() > 0:
        print("\n⚠️ WARNING: Some bills still marked as pending!")
    
    # Analyze failure reasons
    print(f"\nFailure reasons breakdown:")
    failure_counts = df[df['failure_reason'].notna()]['failure_reason'].value_counts()
    for reason, count in failure_counts.items():
        print(f"- {reason}: {count} bills ({count/len(df)*100:.1f}%)")
    
    # Additional analysis
    print(f"\nAction count analysis for failed bills:")
    failed_bills = df[df['passed'] == 0]
    action_ranges = [
        (1, 1, "Only introduced"),
        (2, 2, "Introduced + referred"),
        (3, 5, "Limited committee action"),
        (6, 10, "Some legislative progress"),
        (11, 20, "Significant progress"),
        (21, 100, "Extensive activity")
    ]
    
    for min_actions, max_actions, description in action_ranges:
        count = len(failed_bills[(failed_bills['action_count'] >= min_actions) & 
                                (failed_bills['action_count'] <= max_actions)])
        if count > 0:
            print(f"- {description}: {count} bills")
    
    return df

def create_enhanced_training_set(df):
    """
    Create training dataset with enhanced failure identification
    """
    
    # Only include bills with known outcomes
    training_df = df[df['passed'] != -1].copy()
    
    print(f"\n" + "="*60)
    print("ENHANCED TRAINING DATASET")
    print("="*60)
    print(f"Total bills for training: {len(training_df)}")
    print(f"Pass rate: {(training_df['passed'] == 1).mean()*100:.1f}%")
    
    # Analyze by bill type
    print("\nOutcomes by bill type:")
    outcome_by_type = pd.crosstab(training_df['bill_type'], training_df['passed'], normalize='index') * 100
    print(outcome_by_type.round(1))
    
    # Analyze by sponsor party
    print("\nOutcomes by sponsor party:")
    outcome_by_party = pd.crosstab(training_df['sponsor_party'], training_df['passed'], normalize='index') * 100
    print(outcome_by_party.round(1))
    
    # Save enhanced training dataset
    training_df.to_csv('../data/bills_118th_congress_training_enhanced.csv', index=False)
    print(f"\nSaved enhanced training data to bills_118th_congress_training_enhanced.csv")
    
    # Also save the full dataset with failure reasons
    df.to_csv('../data/bills_118th_congress_full_enhanced.csv', index=False)
    
    return training_df

def analyze_bill_patterns(df):
    """
    Analyze patterns in passed vs failed bills
    """
    
    print(f"\n" + "="*60)
    print("BILL PATTERN ANALYSIS")
    print("="*60)
    
    passed_bills = df[df['passed'] == 1]
    failed_bills = df[df['passed'] == 0]
    
    print(f"\nPassed bills characteristics:")
    print(f"- Average action count: {passed_bills['action_count'].mean():.1f}")
    print(f"- Average cosponsor count: {passed_bills['cosponsor_count'].mean():.1f}")
    print(f"- Bipartisan rate: {passed_bills['is_bipartisan'].mean()*100:.1f}%")
    print(f"- Average days to pass: {passed_bills['days_since_introduction'].mean():.0f}")
    
    print(f"\nFailed bills characteristics:")
    print(f"- Average action count: {failed_bills['action_count'].mean():.1f}")
    print(f"- Average cosponsor count: {failed_bills['cosponsor_count'].mean():.1f}")
    print(f"- Bipartisan rate: {failed_bills['is_bipartisan'].mean()*100:.1f}%")
    print(f"- Average days before failure: {failed_bills['days_since_introduction'].mean():.0f}")
    
    # Key differences
    print(f"\nKey differences (passed vs failed):")
    print(f"- Action count: {passed_bills['action_count'].mean():.1f} vs {failed_bills['action_count'].mean():.1f}")
    print(f"- Cosponsor count: {passed_bills['cosponsor_count'].mean():.1f} vs {failed_bills['cosponsor_count'].mean():.1f}")
    print(f"- Committee count: {passed_bills['committee_count'].mean():.1f} vs {failed_bills['committee_count'].mean():.1f}")

def main():
    """
    Main function to combine CSVs and identify failures
    """
    
    # Check if we need to combine or if full file already exists
    # I ran the API file acquisition on two computers and got two CSVs, so I needed to combine
    if os.path.exists('bills_118th_congress_full.csv'):
        print("Found existing combined file")
        df = pd.read_csv('bills_118th_congress_full.csv')
    else:
        # Combine the two CSVs
        df = combine_congress_csvs()
    
    # Identify dead bills
    df = identify_dead_bills(df)
    
    # Create enhanced training set
    training_df = create_enhanced_training_set(df)
    
    # Analyze patterns
    analyze_bill_patterns(df)
    
    print("\n✅ Processing complete!")
    print("\nUse 'bills_118th_congress_training_enhanced.csv' for model training")
    print("This should have a much more balanced dataset with realistic failure identification")

if __name__ == "__main__":
    print("="*60)
    print("COMBINING CSVS AND IDENTIFYING COMMITTEE DEATHS")
    print("="*60)
    print("\nPlease ensure your CSV files are named:")
    print("- ../data/bills_118th_congress_house.csv (House bills)")
    print("- ../data/bills_118th_congress_senate_joint.csv (Senate + Joint Resolutions)")
    print("\nOr if you already have a combined file:")
    print("- ../data/bills_118th_congress_full.csv")
    print("="*60)
    
    main()

COMBINING CSVS AND IDENTIFYING COMMITTEE DEATHS

Please ensure your CSV files are named:
- ../data/bills_118th_congress_house.csv (House bills)
- ../data/bills_118th_congress_senate_joint.csv (Senate + Joint Resolutions)

Or if you already have a combined file:
- ../data/bills_118th_congress_full.csv
Found existing combined file

IDENTIFYING FAILED BILLS FROM ENDED 118TH CONGRESS
Current date: 2025-07-26
118th Congress ended: 2025-01-03
Days since Congress ended: 204

Original outcome distribution:
Passed: 274
Failed: 25
Pending: 16266

🔴 118th Congress has ended - all pending bills are now FAILED

Categorizing failure reasons...

Final outcome distribution:
Passed: 274 (1.7%)
Failed: 16291 (98.3%)
Pending: 0 (should be 0)

Failure reasons breakdown:
- died_in_committee: 14461 bills (87.3%)
- stalled_in_process: 1348 bills (8.1%)
- failed_to_complete: 457 bills (2.8%)

Action count analysis for failed bills:
- Introduced + referred: 4911 bills
- Limited committee action: 8749 bills
- S